### Importando bibliotecas 

In [2]:
import cvxpy as cp
import numpy as np
import mosek



### Definindo problema de otimização: variáveis e constraints

In [3]:
vars = 2

# Instanciando o vetor de variáveis de decisão
x = cp.Variable(shape=vars, boolean=False)

# Criando as matrizes e vetores que defininem as restrições do problema
inequalityMatrix = np.array([[2, 1], [1, 3]])
inequalityVector = np.array([1, 1])

positivityMatrix = np.array([[1, 0], [0, 1]])
positivityVector = np.array([0, 0])

# Forma matricial das 3 funções objetivo do exercício
objFunction1 = np.array([1, 1])
objFunction2 = np.array([-1, -1])
objFunction3 = np.array([1, 0])

# Vetor de constraints para o cvxpy
constraints = [inequalityMatrix @ x >= inequalityVector, positivityMatrix @ x >= positivityVector]  # Example of inequality constraints



### Definindo função objetvio e resolvendo o problema

In [7]:
# Para cada uma das funções objetivo, instanciar um problem do cvxpy, resolvê-lo, e printar resultado

objective = cp.Minimize(objFunction1 @ x)
problem = cp.Problem(objective=objective, constraints=constraints)
print(problem)
print("Starting optimization run for f1 - MOSEK")
resultadoCplex = problem.solve(solver="MOSEK", verbose=True)
if(resultadoCplex != None):
    optimalPoint = x.value
    optimum = objFunction1 * x.value
    print("f1: Optimum of {} found at x = {}".format(resultadoCplex, optimalPoint))
else:
    print("Infeasible or unbounded problem.")

print("")
objective = cp.Minimize(objFunction2 @ x)
problem = cp.Problem(objective=objective, constraints=constraints)
print("Starting optimization run for f2 - MOSEK")
resultadoCplex = problem.solve(solver="MOSEK", verbose=False)
if(resultadoCplex != None):
    optimalPoint = x.value
    optimum = objFunction2 * x.value
    print("f2: Optimum of {} found at x = {}".format(resultadoCplex, optimalPoint))
else:
    print("Infeasible or unbounded problem.")

print("")
objective = cp.Minimize(objFunction3 @ x)
problem = cp.Problem(objective=objective, constraints=constraints)
print("Starting optimization run for f3 - MOSEK")
resultadoCplex = problem.solve(solver="MOSEK", verbose=False)
if(resultadoCplex != None):
    optimalPoint = x.value
    optimum = objFunction3 * x.value
    print("f3: Optimum of {} found at x = {}".format(resultadoCplex, optimalPoint))
else:
    print("Infeasible or unbounded problem.")

# note que nesse último caso, o solver nos repassa um ponto específico de mínimo com o menor valor numérico
# da variável livre x2. No entanto, qualquer ponto da semi-reta x2 >= 1 seria igualmente válido. 


minimize [1. 1.] @ var1
subject to [1. 1.] <= [[2.00 1.00]
 [1.00 3.00]] @ var1
           [0. 0.] <= [[1.00 0.00]
 [0.00 1.00]] @ var1
Starting optimization run for f1 - MOSEK
                                     CVXPY                                     
                                     v1.5.1                                    
(CVXPY) Jun 17 05:55:06 PM: Your problem has 2 variables, 4 constraints, and 0 parameters.
(CVXPY) Jun 17 05:55:06 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 17 05:55:06 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 17 05:55:06 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Jun 17 05:55:06 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation           

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

## Programação Semi-Definida:

In [8]:
# No caso da programção semi-definida, seguimos os mesmo passos do exercício anterior
# Aqui no entanto, nossas variáveis serão componentes de uma matriz simétrica

# código feio horroroso para construir as matrizes do problema
A = np.zeros((4,4))
B = np.zeros((4,4))
Id = np.identity(4)

A[0,3] = A[1,2] = A[2,3] = A[3,0] = B[0,0] = B[3,3] = 1
B[1,1] = B[2,2] = -1

# instanciando uma matriz simétrica como variável de decisão
X = cp.Variable((4,4), symmetric=True)

# criando lista de constraints para o cvxpy
constraints = [cp.trace(B @ X) == 1, cp.trace(X) == 1, X >> 0]

# instanciando e resolvendo problema 
objective = cp.Maximize(cp.trace(A @ X))
prob = cp.Problem(objective=objective, constraints=constraints)
prob.solve(solver="MOSEK")

# mostrando resultados
# truncamos o resultado final pois haviam pequenos devios nos valores, muito provavelmente advindos de erros de ponto flutuante
print("The optimal value is", '%.3f'%(prob.value))
print("A solution X is")
print(X.value.round(3))

The optimal value is 1.000
A solution X is
[[ 0.5  0.   0.   0.5]
 [ 0.  -0.   0.   0. ]
 [ 0.   0.   0.   0. ]
 [ 0.5  0.   0.   0.5]]
